# Coadd Catalog Generation & Injection Tutorial Notebook

For the Rubin Science Platform at <a href="https://data.lsst.cloud">data.lsst.cloud</a><br>
Data Release: <a href="https://dp1.lsst.io/">Data Preview 1 (DP1)</a> <br>

**Learning Objective:** To learn how to generate time-varied visit injection catalogs of stamps and average them to produce appropriate injected coadd images. 

**LSST Data Products:** `visit_image`, `deep_coadd`

**Packages:** `lsst.daf.butler`, `lsst.rsp`, `lsst.source.injection`, `lsst.afw.display`, `lsst.afw.image`

**Credit:** This notebook was created by Dhruv Sharma using code developed by Shenming Fu (SLAC National Accelerator Laboratory)

## 1. Introduction

The LSST Camera is made to photograph patches of sky one after the other in quick captures called `visit_image`s. These photos provide a wide, rapidly imaged view of the night sky, but are often shallow and noisy. Even more useful are `deep_coadd` images, which are created by compositing multiple `visit_image` captures of the same patch of sky together using an averaging process to reduce noise.

To help us perform time-delay cosmography using Rubin data, we must be able to simulate time-varying sources and represent them in both `visit_image` and `deep_coadd` injections. For phenomena like strongly lensed AGN, we often inject a "stamp" in the form of a `.fits` file into the images that contains a 2D matrix representation of the SL-AGN. In this tutorial, we will demonstrate the following:

- Generating multiple stamp catalogs corresponding to different `visit_image`s, where the stamps vary from image to image
- Rotating stamps to appropriately inject them into visits (which are not aligned to the ra,dec grid by default) and keep their orientation consistent between captures.
- Compositing multiple stamps to produce a coadded stamp that represents the average of a source over time and injecting a catalog of such averaged stamps into a `deep_coadd`

### 1.1. Import Packages

Import all the necessary packages for this notebook.

In [ ]:
from lsst.daf.butler import Butler
import lsst.afw.display as afwDisplay
from astropy.table import Table, vstack
from lib.tools import *
from lib.stamp import *
from lib.inject import *
import matplotlib.pyplot as plt
import numpy as np
from lsst.afw.image import ExposureF
%matplotlib inline

### 1.2. Using the Butler to retrieve visit images

Create a new instance of the Butler configured to query DP1 data from the LSSTComCam. Also ensure that our plots are made and displayed in matplotlib using `afwDisplay.setDefaultBackend`.

In [ ]:
butler = Butler("dp1", collections="LSSTComCam/DP1")
afwDisplay.setDefaultBackend("matplotlib")

Use the Butler's `query_datasets` method to find `visit_image`s corresponding to the  band and ra,dec (right-ascension, declination) coordinates given in the "bind" of the formatted string, in this case:
>ra = 37.93, dec = 6.93 <br>
>band = 'r'

There can and often may be be multiple visit images covering the same ra,dec position, as seen by printing the length of `visit_data`.

In [ ]:
band = 'r'
ra = 37.93
dec = 6.93
query = "band.name = :band AND " \
        "visit_detector_region.region OVERLAPS POINT(:ra, :dec)"
bind_params = {"band": band, "ra": ra, "dec": dec}
visit_data = butler.query_datasets("visit_image", 
                                   where=query,
                                   bind=bind_params,
                                   order_by=["visit.timespan.begin"])
len(visit_data)

We hope to inject sources into multiple `visit_image`s and a `deep_coadd` image, the latter of which captures a fixed patch of sky. So, we should choose visits that share significant overlap with the coadd. In this notebook, our time series will consist of the 1st, 4th, and 7th images from the `visit_data` dataset. Retrieve all images.

In [ ]:
visit_1 = butler.get(visit_data[0])
visit_2 = butler.get(visit_data[3])
visit_3 = butler.get(visit_data[6])

### 1.3. Using the Butler to Retrieve Coadd Images

Let's also get the `deep_coadd` image corresponding to the patch of sky containing the ra,dec coordinate we queried for.

In [ ]:
query = "band.name = :band AND " \
        "patch.region OVERLAPS POINT(:ra, :dec)"
coadd_data = butler.query_datasets("deep_coadd", 
                                   where=query,
                                   bind=bind_params)
coadd = butler.get(coadd_data[0])

### 1.4. Generating Rotated Stamps

Our `fig` folder contains a few stamp files labeled `system_1_X.fits`, where X is 0, 1, or 2 corresponding to the 3 visits we retrieved. Just as each `visit_image` captures the sky at a different moment in time, each of these visits has its own stamp representing the SL-AGN system at that time. 

Now use the `make_rotated_stamp` method from `lib/stamp.py` to save both a version of the stamp with a synthetic wcs (World Coordinate System), as well as a rotated image of the stamp to the `fig` folder. This rotation is important as `visit_image` captures are not aligned to the ra,dec grid as `deep_coadd` images are. To make sure that stamps are always oriented the same regardless of how the `visit_image` is taken, we rotate them by what's called the _boresight angle_ for each visit.

In [ ]:
make_rotated_stamp(visit_1, 'fig/system_1_0.fits')
make_rotated_stamp(visit_2, 'fig/system_1_1.fits')
make_rotated_stamp(visit_3, 'fig/system_1_2.fits')

Use the `plt` code below to see how each stamp looks before and after rotating by its respective `visit_image`'s boresight angle.

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(3, 4), dpi=200)

def show_image(ax, filename, title):
    image = ExposureF.readFits(filename).image
    array = image.getArray()

    # Display with scaling similar to afwDisplay
    im = ax.imshow(array, origin='lower', vmin=-20, vmax=150, cmap='gray')
    ax.set_title(title, fontsize=6)
    ax.set_xticks([])
    ax.set_yticks([])

for i in range(3):
    show_image(ax[i][0], f'fig/system_1_{i}_wcs.fits', 'system_1_'+str(i)+'.fits')
    show_image(ax[i][1], f'fig/rotated_system_1_{i}_wcs.fits', 'rotated_system_1_'+str(i)+'_wcs.fits')

plt.show()

## 2. Injecting Sources into Time Series of Visit Images

### 2.1. Generating Multiple Injection Catalogs

To populate each `visit_image` with our SL_AGN stamps, we need a way to store an ra,dec coordinate for each source as well as their time-varying magnitudes. So, let's first define an array `source_list` of tuples describing each source. Each entry contains a single source's ra,dec coordinate alongside an array of magnitudes for it, and we generate these values pseudo-randomly around the ra,dec coordinate we chose earlier and with magnitudes between a specified range.

In [ ]:
# source list details time-varying brightnesses of sources
num_sources = 10
num_times = 3
source_list = []    
for i in range(num_sources):
    source_list.append((np.random.uniform(ra-0.03, ra+0.03), 
                        np.random.uniform(dec-0.03, dec+0.03), 
                        np.random.uniform(12,17, size=num_times)))

The above list isn't in the same format as injection catalogs, which are stored as Tables, so we need to transfer the information for each source to a new structure. Create a list `inj_catalogs` of Tables, one for each `visit_image` and populate it with sources using the data from `source_list` we just generated.

In [ ]:
inj_catalogs = []
for i in range(num_times):
    inj_catalogs.append(Table())

In [ ]:
for time in range(num_times):
    for source in range(num_sources):
        inj_catalogs[time] = vstack([inj_catalogs[time], Table(
            {'injection_id': [source],
                'ra': [source_list[source][0]],
                'dec': [source_list[source][1]],
                'source_type': ['Stamp'],
                'mag': [source_list[source][2][time]],
                'stamp': ['fig/rotated_system_1_%s_wcs.fits'%time],
             # if you've generated enough stamps for each source to be unique, use 'fig/system_%s_%s_wcs.fits'%(source,time)
            }
        )])

### 2.2. Injecting into Visit Images

Now use the `visit_inject_stamp` method for each `visit_image` and its corresponding Table entry in our array to create 3 injected images containing the same catalogs with varying magnitudes.

In [ ]:
inj_visit_1 = visit_inject_stamp(visit_1, inj_catalogs[0])
inj_visit_2 = visit_inject_stamp(visit_2, inj_catalogs[1])
inj_visit_3 = visit_inject_stamp(visit_3, inj_catalogs[2])

View all 3 injected `visit_image`s next to one another. Notice how the injected sources have the same position and orientation relative to one another between captures, just as we intended!

In [ ]:
fig = plt.figure()
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(inj_visit_1.image)
plt.title("inj_visit_1: ")
plt.show()

In [ ]:
fig = plt.figure()
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(inj_visit_2.image)
plt.title("inj_visit_2: ")
plt.show()

In [ ]:
fig = plt.figure()
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(inj_visit_3.image)
plt.title("inj_visit_3: ")
plt.show()

## 3. Injecting Sources into Coadd Images

With our `visit_image` injections complete, let's now inject these sources into our `deep_coadd` image. Recall that for these, we should try to imitate the process used to produce coadds and create an "averaged" stamp of our source in the different visit images that we inject into the image. 

### 3.1. Generating Coadd Stamp Catalogs

Use the `get_coadd_stamp` method to generate and view a stamp averaging the various stamps already generated for the system index provided, 1 in this case.

In [ ]:
get_coadd_stamp(1)

The code below creates a new list containing the average magnitude of each source based on the magnitudes they had in the 3 `visit_image` injection catalogs we generated previously.

In [ ]:
average_list = []

for source in range(len(source_list)):
    sum = 0.0
    for mag in range(len(source_list[source][2])):
        sum += source_list[source][2][mag]
    avg = sum / len(source_list[source][2])
    average_list.append(avg)

Then, similar to how we converted `source_list` into a list of catalogs, we'll turn `average_list` into a single catalog containing the same sources from our `visit_image` injections, just with magnitudes that are averaged over all 3 images to be injected into our `deep_coadd`.

In [ ]:
avg_inj_catalog = Table()

for mag in range(len(average_list)):
    avg_inj_catalog = vstack([avg_inj_catalog, Table(
        {'injection_id': [mag],
            'ra': [source_list[mag][0]],
            'dec': [source_list[mag][1]],
            'source_type': ['Stamp'],
            'mag': [average_list[mag]],
            'stamp': ['fig/system_1_coadd.fits'],
        }
    )])

### 3.2. Injecting Coadd Stamp Catalogs

Finally, inject our averaged catalog into the `deep_coadd` image we retrieved earlier using the `template_inject_stamp` method.

In [ ]:
inj_coadd = template_inject_stamp(coadd, avg_inj_catalog)

View the injected `deep_coadd` image.

In [ ]:
fig = plt.figure()
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(inj_coadd.image)
plt.title("inj_coadd: ")
plt.show()

## 4. Visualizing Visit and Coadd Image Regions

As a final task, run the below code to view a diagram visualizing the `visit_image` regions alongside the patch captured in our `deep_coadd`, as well as the area we injected our sources into.

In [ ]:
def pixel_to_degrees(n):
    return 0.2 * n / 3600

In [ ]:
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
import numpy as np
from astropy.wcs import WCS

# Create figure and axes
fig = plt.figure()
ax = plt.subplot(projection=WCS(coadd.getWcs().getFitsMetadata()))
plt.xlabel("RA [deg]")
plt.ylabel("DEC [deg]")

# Coadd Rectangle
coadd_cen = coadd.wcs.pixelToSky(coadd.getBBox().getCenter())
coadd_ra = coadd_cen.getRa().asDegrees()
coadd_dec = coadd_cen.getDec().asDegrees()
coadd_width = pixel_to_degrees(coadd.width)
coadd_height = pixel_to_degrees(coadd.height)
coadd_rect = patches.Rectangle(
    (coadd_ra - coadd_width / 2, coadd_dec - coadd_height / 2),  # lower-left corner
    coadd_width,
    coadd_height,
    angle=0,
    rotation_point = 'center',
    linewidth=2,
    edgecolor='blue',
    facecolor='none'
)

# Visit 1 Rectangle
visit_cen = visit_1.wcs.pixelToSky(visit_1.getBBox().getCenter())
visit_ra = visit_cen.getRa().asDegrees()
visit_dec = visit_cen.getDec().asDegrees()
visit_width = pixel_to_degrees(visit_1.width)
visit_height = pixel_to_degrees(visit_1.height)
visit_angle = -visit_1.visitInfo.getBoresightRotAngle().asDegrees()
visit_rect = patches.Rectangle(
    (visit_ra - visit_width / 2, visit_dec - visit_height / 2),  # lower-left corner
    visit_width,
    visit_height,
    angle=visit_angle,
    rotation_point = 'center',
    linewidth=2,
    edgecolor='red',
    facecolor='none'
)

# Visit 2 Rectangle
visit_2_cen = visit_2.wcs.pixelToSky(visit_2.getBBox().getCenter())
visit_2_ra = visit_2_cen.getRa().asDegrees()
visit_2_dec = visit_2_cen.getDec().asDegrees()
visit_2_width = pixel_to_degrees(visit_2.width)
visit_2_height = pixel_to_degrees(visit_2.height)
visit_2_angle = -visit_2.visitInfo.getBoresightRotAngle().asDegrees()
visit_2_rect = patches.Rectangle(
    (visit_2_ra - visit_2_width / 2, visit_2_dec - visit_2_height / 2),  # lower-left corner
    visit_2_width,
    visit_2_height,
    angle=visit_2_angle,
    rotation_point = 'center',
    linewidth=2,
    edgecolor='red',
    facecolor='none'
)

# Visit 3 Rectangle
visit_3_cen = visit_3.wcs.pixelToSky(visit_3.getBBox().getCenter())
visit_3_ra = visit_3_cen.getRa().asDegrees()
visit_3_dec = visit_3_cen.getDec().asDegrees()
visit_3_width = pixel_to_degrees(visit_3.width)
visit_3_height = pixel_to_degrees(visit_3.height)
visit_3_angle = -visit_3.visitInfo.getBoresightRotAngle().asDegrees()
visit_3_rect = patches.Rectangle(
    (visit_3_ra - visit_3_width / 2, visit_3_dec - visit_3_height / 2),  # lower-left corner
    visit_3_width,
    visit_3_height,
    angle=visit_3_angle,
    rotation_point = 'center',
    linewidth=2,
    edgecolor='red',
    facecolor='none'
)

# Injection Region
inj_region_rect = patches.Rectangle(
    (37.93-0.03, 6.93-0.03),
    0.06,
    0.06,
    angle=0,
    rotation_point = 'center',
    linewidth=2,
    edgecolor='none',
    facecolor='gainsboro'
)


# Add rectangles to plot
ax.add_patch(coadd_rect)
ax.add_patch(inj_region_rect)
ax.add_patch(visit_rect)
ax.add_patch(visit_2_rect)
ax.add_patch(visit_3_rect)
ax.invert_xaxis()

ax.set_xlim(coadd_ra + 1.2*coadd_width, coadd_ra - coadd_width)
ax.set_ylim(coadd_dec - 1.4*coadd_height, coadd_dec + 0.7*coadd_height)
ax.set_aspect('equal')
plt.legend(["coadd", "visit", "injection region"], loc="lower right")
plt.grid(ls=':')